In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import expit
from sklearn.metrics import accuracy_score

from NeuralNet import NeuralNet, PCA, Dense, softmax

np.set_printoptions(suppress=True, precision=2)

In [ ]:
import urllib.request

training = (
    "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
    "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"
)

testing = (
    "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
    "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"
)

def dl_mnist(ds, prefix):
    img_url, lab_url = ds
    print(f"Downloading {prefix}")
    urllib.request.urlretrieve(img_url, f'./{prefix}-img')
    urllib.request.urlretrieve(lab_url, f'./{prefix}-lab')
    print("Done")

In [ ]:
dl_mnist(training, "train")

In [ ]:
import struct
import gzip

def print_data(x):
    plt.imshow(x.reshape(28, 28), cmap='Greys')
    plt.axis('off')
    
def read_dataset(imgname, labname):
    X = []
    y = []
    with gzip.open(imgname, "rb") as img, gzip.open(labname, "rb") as labs:
        img_header = struct.unpack(">4i", img.read(16))
        lab_header = struct.unpack(">2i", labs.read(8))
        
        img_size = img_header[2] * img_header[3]
        
        for i in range(img_header[1]):
            image = struct.unpack(f"{img_size}B", img.read(img_size))
            label = struct.unpack("B", labs.read(1))
            X.append(image)
            y.append(label[0])
        X = np.array(X) / 255
    return X, np.array(y)

In [ ]:
X, y = read_dataset("train-img", "train-lab")
yt = np.array([[1 if i == x else 0 for i in range(10)] for x in y])

In [ ]:
dl_mnist(testing, "test")

In [ ]:
X_test, y_test = read_dataset("test-img", "test-lab")
yt_test = np.array([[1 if i == x else 0 for i in range(10)] for x in y_test])

In [ ]:
NN = NeuralNet()

In [ ]:
NN.append(Dense(10, 784, learning_rate=0.1))
#NN.append(Dense(10, 10, learning_rate=0.1))
NN.append(Dense(10, 10, activation=softmax, learning_rate=0.1))

In [ ]:
loss = NN.train(X, yt, epoch=200, batch=100, verbose=True)

In [ ]:
plt.plot(loss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
example = 110
print_data(X_test[example])
NN.forward(X_test[example])

In [ ]:
accuracy_score(y_test, np.argmax(NN.forward(X_test), axis=1))